In [2]:
from google.colab import files
files.upload()

!mkdir ~p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d akensert/bert-base-uncased-huggingface-transformer

!kaggle competitions download -c tweet-sentiment-extraction

import zipfile
with zipfile.ZipFile('./bert-base-uncased-huggingface-transformer.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

import zipfile
with zipfile.ZipFile('./train_csv.zip', 'r') as zip_ref:
    zip_ref.extractall('./')
    print("Done...!")

Done...!


In [3]:
import numpy as np 
import pandas as pd
import os

In [5]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
# pip install transformers
from transformers import *
import tokenizers

print('TF version',tf.__version__)

TF version 2.2.0


In [6]:
MAX_LEN = 114
PATH = './'
import transformers

tokenizer = tokenizers.BertWordPieceTokenizer(
    f"bert-base-uncased-vocab.txt",
    lowercase=True
)


train = pd.read_csv('./train.csv').fillna('')
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [7]:
 tokenizer.encode('positive negative neutral').ids
 sentiment_id = {'positive': 3893, 'negative': 4997, 'neutral': 8699}

In [8]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    #if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+4] = [101] + enc.ids + [102] + [s_tok] + [102]
    attention_mask[k,:len(enc.ids)+4] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

In [9]:
test = pd.read_csv('./test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+4] = [101] + enc.ids + [102] + [s_tok] + [102]
    attention_mask_t[k,:len(enc.ids)+4] = 1

In [10]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    bert_conf = BertConfig()
    #bert_model = transformers.BertModel(PATH+'pretrained-roberta-base.h5',config=config)
    bert_model = TFBertModel.from_pretrained('./bert-base-uncased-tf_model.h5', config=bert_conf)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Conv1D(1,1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(1,1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [11]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [12]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=2,shuffle=True,random_state=777)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
        
    sv = tf.keras.callbacks.ModelCheckpoint(
        '%s-bert-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')
        
    model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
        epochs=3, batch_size=32, verbose=DISPLAY, callbacks=[sv],
        validation_data=([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], 
        [start_tokens[idxV,], end_tokens[idxV,]]))
    
    print('Loading model...')
    model.load_weights('%s-bert-%i.h5'%(VER,fold))
    
    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-1:b])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()

#########################
### FOLD 1
#########################


Some weights of the model checkpoint at ./bert-base-uncased-tf_model.h5 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at ./bert-base-uncased-tf_model.h5.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/3
430/430 [==============================] - ETA: 0s - loss: 2.7045 - activation_loss: 1.3715 - activation_1_loss: 1.3330
Epoch 00001: val_loss improved from inf to 2.08073, saving model to v0-bert-0.h5
430/430 [==============================] - 786s 2s/step - loss: 2.7045 - activation_loss: 1.3715 - activation_1_loss: 1.3330 - val_loss: 2.0807 - val_activation_loss: 1.0383 - val_activation_1_loss: 1.0424
Epoch 2/3
430/430 [==============================] - ETA: 0s - loss: 1.9996 - activation_loss: 1.0138 - activation_1_loss: 0.9859
Epoch 00002: val_loss improved from 2.08073 to 1.97456, saving model to v0-bert-0.h5
430/430 [==============================] - 784s 2s/step - loss: 1.9996 - activation_loss: 1.0138 - activation_1_loss: 0.9859 - val_loss: 1.9746 - val_activation_loss: 0.9914 - val_activation_1_loss: 0.9832
Epoch 3/3
430/430 [==============================] - ETA: 0s - loss: 1.6344 - activation_loss: 0.8280 - activation_1_loss: 0.8064
Epoch 00003: val_loss did not im

Some weights of the model checkpoint at ./bert-base-uncased-tf_model.h5 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at ./bert-base-uncased-tf_model.h5.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/3
430/430 [==============================] - ETA: 0s - loss: 2.7819 - activation_loss: 1.3920 - activation_1_loss: 1.3899
Epoch 00001: val_loss improved from inf to 2.11740, saving model to v0-bert-1.h5
430/430 [==============================] - 788s 2s/step - loss: 2.7819 - activation_loss: 1.3920 - activation_1_loss: 1.3899 - val_loss: 2.1174 - val_activation_loss: 1.0611 - val_activation_1_loss: 1.0563
Epoch 2/3
430/430 [==============================] - ETA: 0s - loss: 2.0474 - activation_loss: 1.0204 - activation_1_loss: 1.0270
Epoch 00002: val_loss improved from 2.11740 to 2.01223, saving model to v0-bert-1.h5
430/430 [==============================] - 785s 2s/step - loss: 2.0474 - activation_loss: 1.0204 - activation_1_loss: 1.0270 - val_loss: 2.0122 - val_activation_loss: 1.0298 - val_activation_1_loss: 0.9824
Epoch 3/3
430/430 [==============================] - ETA: 0s - loss: 1.6710 - activation_loss: 0.8359 - activation_1_loss: 0.8351
Epoch 00003: val_loss did not im

In [13]:
print('>>>> OVERALL 3Fold CV Jaccard =',np.mean(jac))

>>>> OVERALL 3Fold CV Jaccard = 0.5096912802084917


In [17]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-1:b])
    st.replace("##","")
    all.append(st)

In [18]:
test['selected_text'] = all
test[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test.sample(25)

,textID,text,sentiment,selected_text
3290,edef02e047,_7 Wow that`s a big list...lol. I would be happy if I go...,neutral,7 wow that ` s a big list... lol. i would be happy if i ...
584,4f7cba2eff,"ach, probably not... they are labeled as MI-5, which I ...",neutral,"ach, probably not... they are labeled as mi - 5, which i..."
1476,d72748e39c,u never sent me carrie. ur an ****. but an **** that i ...,negative,miss
661,bcf13877f7,Good morning everyone,positive,good morning everyone
2027,7907132523,In bed with 2 girls,neutral,in bed with 2 girls
2004,bfb0644d9e,I love Dawn Chorus noisier the better miss Church Bells...,neutral,love dawn chorus noisier the better miss church bells to...
1259,1900a6978a,Idk i cant do ****,negative,idk i cant do * * * *
1725,07c4438a0a,plus I just deposited tax return cheque but for first t...,negative,clear immediately
1143,4d7def819d,"Aw, sorry E. :/ I hope it looks up for you (lame ear) A...",neutral,"aw, sorry e. : / i hope it looks up for you ( lame ear )..."
2390,f765bed159,oh mann... Me likey that!! But sadly I`m not bein aucti...,negative,sadly i ` m not bei


In [19]:
pd.read_csv('submission.csv')

,textID,selected_text
0,f87dea47db,last session of the day http : / / twitpic. com / 67ez
1,96d74cb729,##weeps in china
2,eee518ae67,shame!
3,01082688c6,happy bday!
4,33987a8ee5,##w75p - i like
...,...,...
3529,e5f0e6ef4b,tired but i can
3530,416863ce47,for the net
3531,6332da480c,depression... he wants to move someplace tropical
3532,df1baec676,videos!
